# Grid Injection

Author: Shu Liu
Last Edited Date: 05/020/2020

## Abstract
This tutorial illustrates how to inject fake psf on a grid of `calexp` image. The injection method is from: https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/insertFakes.py#L456

In [ ]:
import numpy as np
from astropy.table import Table

from lsst.daf.persistence import Butler
import lsst.geom as geom
import lsst.afw.image as afwImage

In [19]:
# Here we copy a set of repos for storing calexp images.
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_17
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_18
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_19
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_20
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_21
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_22
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_23
!cp -r ../GitHub/fake_data ../GitHub/fake_grid/fake_grid_24

In [2]:
# Setup data id
tract, patch, filt = 4639, '0,0', 'r'
visit, detector = 181868, 57
calexp_id = {'visit': visit, 'detector': detector, 'filter': filt}
deepCoadd_id = {'tract': tract, 'patch': patch, 'filter': filt}

In [3]:
# This is the function we use to inject fake psfs to an exposure.

def inject_stars(exposure, photoCalib, x, y, magVar):
    exposure.mask.addMaskPlane("FAKE")
    bitmask = exposure.mask.getPlaneBitMask("FAKE")

    xy = geom.Point2D(x, y)
    psf = exposure.getPsf()

    starIm = psf.computeImage(xy)
    # This is the default value in 'mkFakeStars'
    # https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/insertFakes.py#L458
    calibFluxRadius = 12
    correctedFlux = psf.computeApertureFlux(calibFluxRadius, xy)
    starIm /= correctedFlux
    flux = photoCalib.magnitudeToInstFlux(magVar, xy)
    starIm *= flux
    
    fakeImage = starIm.convertF()
    
    imageBBox = exposure.getBBox()
    imageMI = exposure.maskedImage
    
    interpFakeImage = fakeImage
    interpFakeImBBox = fakeImage.getBBox()
    interpFakeImBBox.clip(imageBBox)
    imageMIView = imageMI.Factory(imageMI, interpFakeImBBox)
    
    if interpFakeImBBox.getArea() > 0:
        clippedFakeImage = interpFakeImage.Factory(interpFakeImage, interpFakeImBBox)
        clippedFakeImageMI = afwImage.MaskedImageF(clippedFakeImage)
        clippedFakeImageMI.mask.set(bitmask)
        imageMIView += clippedFakeImageMI

In [22]:
# This function inject fake PSFs onto a grid of calexp image.
# It also saves the calexp with fakes into a specific repo
def save_fakes(calexp_repo, calexp_id, magVar):
    calexp_butler = Butler(calexp_repo)
    calexp_photoCalib = calexp_butler.get('calexp_photoCalib',  calexp_id)
    calexp_exposure = calexp_butler.get('calexp', dataId=calexp_id)
    for x in range(1000, 3000, 400):
        for y in range(1000, 3000, 400):
            inject_stars(calexp_exposure, calexp_photoCalib, x, y, magVar)
    calexp_butler.put(calexp_exposure, 'calexp', dataId=calexp_id)

In [23]:
# Start injecting fakes
for mag in range(17, 25):
    repo = '/global/u1/s/shl159/GitHub/fake_grid/fake_grid_{}'.format(mag)
    save_fakes(repo, calexp_id, mag)

In [24]:
############################################################
############################################################
##### Then we run imageDifferenceDriver.py in terminal #####
############################################################
############################################################

In [4]:
# This function check the performance of imageDifferenceDrive.py
def check_detecion(repo, data_id, save_path):
    butler = Butler(repo)
    diff_src = butler.get('deepDiff_diaSrc', dataId=calexp_id)
    diff_astropy = diff_src.asAstropy()
    print(f'tot detections: {len(diff_astropy)}')
    row_num = []
    # filter out detections with flags
    for num, src in enumerate(diff_astropy):
        if (src['base_PixelFlags_flag_saturatedCenter'] == 0) \
        and (src['base_PixelFlags_flag_saturated'] == 0) \
        and (src['base_PixelFlags_flag_edge']) == 0 \
        and (src['base_PixelFlags_flag_offimage'] == 0):
            row_num.append(num)
    good_diff_table = diff_astropy[row_num]
    good_diff_table['coord_ra'] = np.rad2deg(good_diff_table['coord_ra'])
    good_diff_table['coord_dec'] = np.rad2deg(good_diff_table['coord_dec'])
    
    print(f'after flag: {len(good_diff_table)}')
    
    check_coord = [(x, y) for x in range(1000, 3000, 400) for y in range(1000, 3000, 400)]
    
    for src in good_diff_table:
        x = src['base_NaiveCentroid_x']
        y = src['base_NaiveCentroid_y']
        ra, dec = src['coord_ra'], src['coord_dec']
        count = 0
        for coord in check_coord:
            if np.abs(x - coord[0]) < 3 and np.abs(y - coord[1]) < 3:
                detect_coord.append((ra, dec))
                detect_radecxy.append((ra, dec, coord[0], coord[1]))
                count += 1
    
    print(f'# of detected fakes {len(detect_coord)}')
    

In [5]:
for mag in range(17, 25):
    print(f'mag: {mag}')
    repo = '/global/u1/s/shl159/GitHub/fake_grid/al_grid_{}'.format(mag)
    save_path = '../GitHub/fake_grid/al_grid_{}.txt'.format(mag)
    check_detecion(repo, calexp_id, save_path)
    print('\n')

mag: 17
tot detections: 155
after flag: 123
# of detected fakes 25


mag: 18
tot detections: 155
after flag: 124
# of detected fakes 25


mag: 19
tot detections: 155
after flag: 124
# of detected fakes 25


mag: 20
tot detections: 155
after flag: 124
# of detected fakes 25


mag: 21
tot detections: 155
after flag: 124
# of detected fakes 25


mag: 22
tot detections: 155
after flag: 124
# of detected fakes 25


mag: 23
tot detections: 155
after flag: 124
# of detected fakes 25


mag: 24
tot detections: 149
after flag: 118
# of detected fakes 19


